<a href="https://colab.research.google.com/github/omniaghazy/Core_X/blob/main/trajectory_Stiffness_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pybullet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybullet: filename=pybullet-3.2.7-cp312-cp312-linux_x86_64.whl size=99873170 sha256=1b5d92bbb79e82355ce3a4e8db769acd8762ae745c8c020a84a44066f170af1e
  Stored in directory: /root/.cache/pip/wheels/72/95/1d/b336e5ee612ae9a019bfff4dc0bedd100ee6f0570db205fdf8
Successfully built pybullet


In [2]:
import pybullet as p
import time
import pybullet_data
import math
import pandas as pd
import os
import numpy as np
from typing import List

print("--- Starting Joint Stiffness Loss Simulation ---")

# ==========================================================
# ⚙️ الـSetup: Connect والـLoad
# ==========================================================
# يجب أن تضعي هنا كود الـSetup الكامل (p.connect، loadURDF، إلخ.)
# أنا أعتبر أن (robotId, controllable_joints, end_effector_link_index)
# تم تعريفهم بنجاح في هذا الجزء.

physicsClient = p.connect(p.DIRECT)
pybullet_data_path = pybullet_data.getDataPath()
p.setAdditionalSearchPath(pybullet_data_path)
p.setGravity(0, 0, -9.81)
p.setRealTimeSimulation(0)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 0.01]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
robotId = p.loadURDF("kuka_iiwa/model.urdf", startPos, startOrientation, useFixedBase=True)

numJoints = p.getNumJoints(robotId)
controllable_joints = [i for i in range(numJoints) if p.getJointInfo(robotId, i)[2] == p.JOINT_REVOLUTE]
end_effector_link_index = 6
joint_to_fail = 4

for joint in controllable_joints:
    p.setJointMotorControl2(robotId, joint, p.VELOCITY_CONTROL, force=0) # لا نحتاج Sensors هنا

# --- 5. إعداد حقن العطل (Stiffness Loss) ---
INITIAL_STIFFNESS = 100.0   # قيمة الصلابة الأولية (قيمة افتراضية كبيرة)
MIN_STIFFNESS = 10.0        # أقل قيمة للصلابة
total_steps = 20000
fault_injection_step = 10000
STIFFNESS_DECREMENT = (INITIAL_STIFFNESS - MIN_STIFFNESS) / (total_steps - fault_injection_step)

data_log: List[List] = []
print(f"✅ Setup Done. Simulation starting for {total_steps} steps (Stiffness Loss).")

# --- 6. اللوب الرئيسي للمحاكاة (توليد البيانات) ---
current_stiffness = INITIAL_STIFFNESS
try:
    for step in range(total_steps):

        # أ. حساب المسار (Trajectory) والـIK
        target_x = 0.6 + 0.2 * math.sin(step * 0.01)
        target_y = 0.0
        target_z = 0.8
        target_position = [target_x, target_y, target_z]
        perfect_commands_list = p.calculateInverseKinematics(robotId, end_effector_link_index, target_position)

        # --- ج. (حقن العطل) ---
        if step >= fault_injection_step:
            if step == fault_injection_step:
                print(f"[STEP {step}] --- FAULT INJECTED! Starting Stiffness Decrease. ---")

            # تقليل قيمة الـStiffness تدريجياً
            current_stiffness = max(current_stiffness - STIFFNESS_DECREMENT, MIN_STIFFNESS)

            # تطبيق الـChange Dynamics
            p.changeDynamics(
                bodyUniqueId=robotId,
                linkIndex=joint_to_fail,
                jointStiffness=current_stiffness
            )

        log_row = [step]
        # --- د. لوب على كل المفاصل للتحكم والتسجيل ---
        for i in range(len(controllable_joints)):
            joint_index = controllable_joints[i]
            target_pos = perfect_commands_list[i]

            # 2. إرسال الأمر للمحاكي (هنا الـController ثابت، لكن الـPhysics بتتغير)
            p.setJointMotorControl2(bodyIndex=robotId, jointIndex=joint_index, controlMode=p.POSITION_CONTROL,
                                    targetPosition=target_pos, positionGain=0.1, velocityGain=0.5)

            # 3. تسجيل البيانات
            joint_state = p.getJointState(robotId, joint_index)
            actual_position = joint_state[0]

            log_row.append(target_pos) # Command Position
            log_row.append(actual_position) # Actual Position (لتقييم الـError)


        log_row.append(current_stiffness) # Stiffness Label
        data_log.append(log_row)
        p.stepSimulation()

except Exception as e:
    print(f"\nAn error occurred during simulation: {e}")
finally:
    # --- 7. إنهاء الاتصال وحفظ البيانات ---
    p.disconnect()
    if data_log:
        columns = ['Step']
        for j in controllable_joints:
            columns.extend([f'Cmd_J{j}', f'Act_J{j}'])
        columns.append('Stiffness_Label')

        df = pd.DataFrame(data_log, columns=columns)
        output_filename = "trajectory_Stiffness_loss_data.csv"
        df.to_csv(output_filename, index=False)
        print(f"\n✅ SUCCESS! Data saved to: {output_filename}")

--- Starting Joint Stiffness Loss Simulation ---
✅ Setup Done. Simulation starting for 20000 steps (Stiffness Loss).
[STEP 10000] --- FAULT INJECTED! Starting Stiffness Decrease. ---

An error occurred during simulation: 'jointStiffness' is an invalid keyword argument for this function

✅ SUCCESS! Data saved to: trajectory_Stiffness_loss_data.csv
